# 获取 Label 页文字和 URL 的对应关系

In [2]:
# !pip3 install requests
import requests
import json
import numpy as np
import random
import math

In [3]:
token = "secret_PtfPQolDWSViYUd08pqg26ab1vseidEoGjf1qFwIrBy"

In [4]:
# Label 页面
database_id = "c861a78fa67d4ea7949b24e62c516d2c"

url = "https://api.notion.com/v1/databases/"+database_id+"/query"

payload = {"page_size": 100}

headers = {
    "Authorization": "Bearer " + token,
    "Accept": "application/json",
    "Notion-Version": "2022-02-22",
    "Content-Type": "application/json"
}

response = requests.request("POST", url, json=payload, headers=headers)

In [5]:
def url_process(url_):
    """
    bad case :

    这 3 个 URL 指向的都是同一个页面

    - https://www.notion.so/706community/up-b3708e2533144b6a9cfcc9b15154fa83
    - [伙伴页面使用的] : https://www.notion.so/b3708e2533144b6a9cfcc9b15154fa83   
    - [接口请求到的]  : https://www.notion.so/up-b3708e2533144b6a9cfcc9b15154fa83  

    UBI-3fcc3ec337f14619a8e83adc1ded68c8
    3fcc3ec337f14619a8e83adc1ded68c8

    所以， 我们要把 类似 ： LGBT-BDSM-43836f2475ef4089a9f886abc230bc4d
    转换成 https://www.notion.so/43836f2475ef4089a9f886abc230bc4d
    这样才能和伙伴页面的数据进行匹配
    """
    tail_ = url_.split('/')[-1]
    url = "https://www.notion.so/" + tail_.split('-')[-1]
    return url

In [6]:
list_ = json.loads(response.text)['results']

label_dic = {}
for i in range(len(list_)):
    try: 
        #print(i, list_[i]['url'])
        #print(i, list_[i]['properties']['Name']['title'][0]['plain_text'])
        url_ = list_[i]['url']
        label = list_[i]['properties']['Name']['title'][0]['plain_text']
        url = url_process(url_)
        label_dic[url] = label
    except: pass

In [7]:
# 这个将来还可以做二级分类。。
label_dic

{'https://www.notion.so/fe6d6e94b94e4673940d0ab1460a4d2a': '技能：Academic/professional writing in English',
 'https://www.notion.so/443f137807fb41928826c0d354161bc9': '技能：个人背景提升/职业规划',
 'https://www.notion.so/521a4c353bf94a99b33464eba0428b34': '知识分享/圆桌对谈/围炉夜话',
 'https://www.notion.so/e404cee69d884bea8887e74505cff34a': '电视剧/美剧/英剧/小语种剧集',
 'https://www.notion.so/5b83ae451fac4c3aa19604b13aa51d8b': '女性权益/女性主义/性别平等',
 'https://www.notion.so/07ad5e6a236547129ed5fee2aba4d552': '戏剧/舞台剧/话剧',
 'https://www.notion.so/2700d98c21e1402a8a99e625d864a6d0': '爱好：健身/户外/团操',
 'https://www.notion.so/a7b005f6ac0c4388b90c6a83e6841991': '实践：读书会/共学',
 'https://www.notion.so/68b9b5e025ed47db91a46a89d078b08a': '艺术：独立电影/文艺电影/戏剧/导演/发行/制片',
 'https://www.notion.so/b3708e2533144b6a9cfcc9b15154fa83': '技能：纪录片/视频播客制作/up主',
 'https://www.notion.so/f4368304918f41d39e9bbaebda42ab36': '社会身份：大学教授/讲师/沙龙嘉宾',
 'https://www.notion.so/0b2d88237d88496fba411ab5258b871a': '学术：建筑学/城市人类学/附近性',
 'https://www.notion.so/0d9347bc94044e779

# 处理 Export 伙伴 Excel 数据

```bash
Export format ： Markdown & CSV
Include databases： Current view
Include content ： No files or imagesv
```

In [8]:
#!pip3 install pandas numpy
import pandas as pd

In [9]:
df = pd.read_csv("/Users/soda/Repository/partner-visualize/data/guys.csv",sep=',')

In [10]:
print(df.columns)

Index(['名字', '自我介绍', '联系方式：Notion', '性别', '学校/机构/公司', '联系方式：微信', '联系方式：Email',
       '🏷️ 标签/分组', '想认识的人或想做的事', '你和706的故事', '🌲 负责  项目/城市', '🌲 协助 项目/城市',
       '🌲 围观  项目/城市', '🌿 提案过的小任务', '🌿 认领过的小任务', '🍃 做过的活动', '🎋 相关指南/工具包', '补充',
       '最近更新时间', '联系方式：Slack', 'ID', '🍃 参加过的活动'],
      dtype='object')


In [11]:
# df['🏷️ 标签/分组'].unique()

In [12]:
"""
  因为有伙伴存在多个标签，尝试行转列：即一行生成多行，每行单独一个标签
"""
cols = ['名字', '自我介绍','学校/机构/公司','联系方式：微信','想认识的人或想做的事']
df__ = df.set_index(cols)["🏷️ 标签/分组"].str.split(', ', expand=True)\
    .stack().reset_index(drop=True, level=-1).reset_index().rename(columns={0: "label"})
df__['label'] = df__['label'].apply(url_process)

In [13]:
df__['label'] = df__['label'].map(label_dic)

In [14]:
df__.head()

,名字,自我介绍,学校/机构/公司,联系方式：微信,想认识的人或想做的事,label
0,Yiy Zhang,www.zhangyiy.com\n艺术家，现工作生活与柏林,NaN,yiyzhang,NaN,社群身份：706notion协作成员
1,何骆,来自云南大理。社会学系研究生在读。本科就读于中文系，对戏剧创作和写作都很感兴趣。后来深感只愿...,北京大学社会学,being_nechi,NaN,人文：社会学/人类学/哲学
2,何骆,来自云南大理。社会学系研究生在读。本科就读于中文系，对戏剧创作和写作都很感兴趣。后来深感只愿...,北京大学社会学,being_nechi,NaN,实践：公共空间/独立书店/社区营造
3,周玉亮,爱思青年公益发展中心 理事长\n中欧校友爱心联盟 副会长\n联合国开发计划署 前项目官员\n...,NaN,NaN,NaN,社会身份：大学教授/讲师/沙龙嘉宾
4,欧阳,曾经的留守儿童，安徽大学毕业，财新媒体前记者，财新传媒为欧阳艳琴提供了30万公益基金创立了“...,实务学堂,NaN,NaN,社会身份：大学教授/讲师/沙龙嘉宾


In [15]:
# id 可以重复，只是会边连起来
"""
对重复的名字进行编码 

  名字后续要作为 ID， 所以不能重复


df['名字'] = df['名字'].astype('str')
df = df[df['名字']!='nan']

names = dict(df.名字.value_counts())

names_dupli = {}
for k,v in names.items():
    if k=='nan': continue
    if v>1: names_dupli[k]=v
        
print(names_dupli)

def operate_dict(dic, key):
    if dic[key] >= 1:
        dic[key] = dic[key] - 1
    if dic[key] == 0: 
        dic.pop(key)

def Add_numbers_at_the_end(name):
    if name in names_dupli.keys():
        end_num = names_dupli[name]
        operate_dict(names_dupli, name)
        return(name+ '_' + str(end_num))
    else:
        return(name)

#operate_dict(names_dupli,'Stella')

df['名字'] = df['名字'].apply(Add_numbers_at_the_end)
df['名字'].value_counts()

"""

"\n对重复的名字进行编码 \n\n  名字后续要作为 ID， 所以不能重复\n\n\ndf['名字'] = df['名字'].astype('str')\ndf = df[df['名字']!='nan']\n\nnames = dict(df.名字.value_counts())\n\nnames_dupli = {}\nfor k,v in names.items():\n    if k=='nan': continue\n    if v>1: names_dupli[k]=v\n        \nprint(names_dupli)\n\ndef operate_dict(dic, key):\n    if dic[key] >= 1:\n        dic[key] = dic[key] - 1\n    if dic[key] == 0: \n        dic.pop(key)\n\ndef Add_numbers_at_the_end(name):\n    if name in names_dupli.keys():\n        end_num = names_dupli[name]\n        operate_dict(names_dupli, name)\n        return(name+ '_' + str(end_num))\n    else:\n        return(name)\n\n#operate_dict(names_dupli,'Stella')\n\ndf['名字'] = df['名字'].apply(Add_numbers_at_the_end)\ndf['名字'].value_counts()\n\n"

# 转 JS 所用的 Json 格式
## 1. 生成 Category list

In [16]:
with open("CategoryList.json", "a") as f:
    content = json.dumps(dict({'cates': list(df__.label.value_counts().keys())}), ensure_ascii=False )
    f.write(content)


## 2. SelectObjList

In [17]:
SelectObj = {"lists":[]}
for item ,num in  df__.label.value_counts().items():
    SelectObj['lists'].append({"value":item ,"label": item, "nums": num })

In [18]:
with open('/Users/soda/Repository/partner-visualize/src/data/SelectObj.json',"w") as f:
    f.write(json.dumps(SelectObj, ensure_ascii=False ))

In [19]:
# 73 个： 有些人没用某些 Label
# with open('./aa.json',"wb") as f:
#json.dumps(dict(df__.label.value_counts()))

In [20]:
df__.to_json(orient="index",force_ascii=False)

'{"0":{"名字":" Yiy Zhang","自我介绍":"www.zhangyiy.com\\n艺术家，现工作生活与柏林","学校\\/机构\\/公司":null,"联系方式：微信":"yiyzhang","想认识的人或想做的事":null,"label":"社群身份：706notion协作成员"},"1":{"名字":" 何骆","自我介绍":"来自云南大理。社会学系研究生在读。本科就读于中文系，对戏剧创作和写作都很感兴趣。后来深感只愿做一个文学的业余爱好者，更感对社会充满困惑，于是研究生时跑路到了社会学，目前关注的领域是历史社会学和政治社会学。喜欢在故纸堆和新鲜田野里拓展自己对他人和社会的理解。\\n","学校\\/机构\\/公司":"北京大学社会学","联系方式：微信":"being_nechi","想认识的人或想做的事":null,"label":"人文：社会学\\/人类学\\/哲学"},"2":{"名字":" 何骆","自我介绍":"来自云南大理。社会学系研究生在读。本科就读于中文系，对戏剧创作和写作都很感兴趣。后来深感只愿做一个文学的业余爱好者，更感对社会充满困惑，于是研究生时跑路到了社会学，目前关注的领域是历史社会学和政治社会学。喜欢在故纸堆和新鲜田野里拓展自己对他人和社会的理解。\\n","学校\\/机构\\/公司":"北京大学社会学","联系方式：微信":"being_nechi","想认识的人或想做的事":null,"label":"实践：公共空间\\/独立书店\\/社区营造"},"3":{"名字":" 周玉亮","自我介绍":"爱思青年公益发展中心 理事长\\n中欧校友爱心联盟 副会长\\n联合国开发计划署 前项目官员\\n曾经担任高校教师多年，后辞职投身公益，创办爱思青年公益发展中心；倡导分享的人生态度，鼓励公益生活化；在青年成长、公益创新、社群培育等领域持续进行探索； 先后入选银杏伙伴计划、国际访问者领导力计划、中欧国际工商学院公益奖学金；先后受邀前往美国、日本、肯尼亚等国家和地区参访和交流；先后获得凤凰卫视、南方周末等主流媒体深度报道。","学校\\/机构\\/公司":null,"联系方式：微信":null,"想认识的人或想做的事":null,"label":"社会身份：大学教授\\/讲师\\/沙龙嘉宾"},"4":{"名字":"

## 3. data_.json

In [21]:
partnerInfo = {
      "nodes": [{
        "id": "Root",
        "group": 1,
        "image": {
          "src": "./images/706.jpeg",
          "width": 300,
          "height": 200
        },
        "type": "Root"
    },
    ],
    "links":    [ 
    ]
}

# test_list = [ "实践：公共空间/独立书店/社区营造", ]

# np.isnan(df_[df_["名字"]=="@雨 陈 "]['自我介绍'].values[0])
# node-category ,
for category in df__.label.value_counts().keys():
#     if category not in test_list:
#         continue
    partnerInfo['nodes'].append({"id": category, "type":"Category"})
    partnerInfo['links'].append({"source": "Root","target": category })

# node-user-parents  / links
# df_ = df__[df__['label'].isin(test_list)]
# df_ = df_.fillna("")
df_ = df__.copy().fillna("")

for index, row in df_.iterrows():
    partnerInfo['nodes'].append({
        "id": row['名字'], 
        "group": random.randint(0,9), 
        "parent": row['label'],
        "description": row['自我介绍'] 
    })

    partnerInfo['links'].append({
        "source": row['label'],
        "target": row['名字']
    })
    
with open('/Users/soda/Repository/partner-visualize/src/data/partner_data.json',"w") as f:
    f.write(json.dumps(partnerInfo, ensure_ascii=False ))

In [23]:
# partnerInfo